<a href="https://colab.research.google.com/github/matheus896/crewai-examples/blob/main/Automatizar%20o%20planejamento%20de%20eventos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install crewai
!pip install litellm
!pip install crewai 'tools'

In [53]:
import os
from crewai import Agent, Task, Crew, LLM, Process
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

In [56]:
# Suas chaves
os.environ["GROQ_API_KEY"] = "SUA_API_KEY"
os.environ ["SERPER_API_KEY"] = "SUA_API_KEY"

# LLM
llm = LLM (
    model = 'groq/llama3-8b-8192',
    temperature=0.2,
    api_key="GROQ_API_KEY"
    )

In [57]:
# Tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [58]:
# Agents

# Agent 1: Coordenador de Eventos
venue_coordinator = Agent(
    role="Coordenador de Eventos",
    goal="Identificar e reservar um local apropriado "
    "com base nos requisitos do evento",
    tools=[search_tool, scrape_tool],
    verbose=True,
    memory=True,
    llm=llm,
    backstory=(
        "Com um senso aguçado de espaço e "
        "entendimento da logística de eventos, "
        "você se destaca em encontrar e garantir "
        "o local perfeito que se ajuste ao tema, "
        "tamanho e restrições orçamentárias do evento."
    )
)

In [59]:
# Agent 2: Gerente de Logística
logistics_manager = Agent(
    role='Gerente de Logística',
    goal=(
       "Gerenciar toda a logística do evento "
        "incluindo bufê e equipamentos"
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    memory=True,
    llm = llm,
    backstory=(
        "Organizado e detalhista, "
        "você garante que todos os aspectos logísticos do evento, "
        "desde o serviço de bufê até a configuração dos equipamentos, "
        "sejam executados de forma impecável para criar uma experiência perfeita."
    )
)

In [60]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Comercializar o evento de forma eficaz e"
         "comunicar-se com os participantes",
    tools=[search_tool, scrape_tool],
    verbose=True,
    memory=True,
    llm = llm,
    backstory=(
        "Criativo e comunicativo, "
        "você cria mensagens convincentes e "
        "se envolve com potenciais participantes "
        "para maximizar a exposição e a participação no evento."
    )
)

In [61]:
# Criação do objeto Pydantic Venue
# Crie uma classe VenueDetails usando o Pydantic BaseModel.
# Os agentes preencherão esse objeto com informações sobre diferentes locais, criando diferentes instâncias dele.

from pydantic import BaseModel
# Define a Pydantic model for venue details
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

In [62]:
""" Criando tarefas
 Ao usar `output_json`, você pode especificar a estrutura da saída desejada.
 Ao usar `output_file`, você pode obter a saída em um arquivo.
 Ao definir `human_input=True`, a tarefa solicitará feedback humano (se você gosta ou não dos resultados) antes de finalizá-la.
"""
venue_task = Task(
    description="Encontre um local em {event_city} "
                "que atenda aos critérios para {event_topic}.",
    expected_output="Todos os detalhes de um local específico "
                    "que você encontrou para acomodar o evento.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",
      # Outputs the venue details as a JSON file
    agent=venue_coordinator
)


In [63]:
# Ao definir `async_execution=True`, isso significa que a tarefa pode ser executada em paralelo com as tarefas que vêm depois dela.

logistics_task = Task(
    description="Coordenar o serviço de bufê e "
                  "os equipamentos para um evento "
                  "com {expected_participants} participantes "
                  "em {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,
    context= [venue_task],
    agent=logistics_manager
)

In [64]:
marketing_task = Task(
    description="Promover o {event_topic} "
                "com o objetivo de engajar pelo menos "
                "{expected_participants} potenciais participantes.",
    expected_output="Relatório sobre as atividades de marketing "
                  "e o engajamento dos participantes formatado em markdown.",
    async_execution=True,
    context= [logistics_task, venue_task],
    output_file="marketing_report.md",  # Gera o relatório em um arquivo de texto
    agent=marketing_communications_agent
)

In [65]:
# Criando a Crew
# **Nota**: Como definimos `async_execution=True` para as tarefas `logistics_task` e `marketing_task`, agora a ordem delas não importa na lista `tasks`.

event_management_crew = Crew(
    agents=[venue_coordinator,
            logistics_manager,
            marketing_communications_agent],

    tasks=[venue_task,
           logistics_task,
           marketing_task],
    verbose=True
)


In [66]:
# Rodando a Crew com os inputs delimitados

event_details = {
    'event_topic': "Conferência de Inovação Tecnológica",
    'event_description': "Um encontro de inovadores em tecnologia "
                         "e líderes do setor "
                         "para explorar tecnologias futuras.",
    'event_city': "São Paulo",
    'tentative_date': "2024-12-23",
    'expected_participants': 500,
    'budget': 20000,
    'venue_type': "Vibra São Paulo"
}

"""**Nota 2**:
- Como está `human_input=True` para algumas tarefas, a execução solicitará sua opinião antes de terminar a execução.
- Quando ele solicitar feedback, use o ponteiro do mouse para clicar primeiro na caixa de texto antes de digitar qualquer coisa.
"""

'**Nota 2**: \n- Como está `human_input=True` para algumas tarefas, a execução solicitará sua opinião antes de terminar a execução.\n- Quando ele solicitar feedback, use o ponteiro do mouse para clicar primeiro na caixa de texto antes de digitar qualquer coisa.\n'

In [ ]:
result = event_management_crew.kickoff(inputs=event_details)

In [68]:
# Exibir o arquivo gerado `venue_details.json`.

import json
from pprint import pprint

with open('venue_details.json') as f:
   data = json.load(f)

pprint(data)

{'address': 'São Paulo',
 'booking_status': 'available',
 'capacity': 500,
 'name': 'InterContinental São Paulo'}


In [ ]:
"""
Exiba o arquivo `marketing_report.md` gerado.
**Nota**: Depois que a execução do `kickoff` for bem-sucedida, aguarde mais 45 segundos para que o arquivo `marketing_report.md` seja gerado.
Se você tentar executar o código abaixo antes de o arquivo ter sido gerado, sua saída será semelhante a:
marketing_report.md
Se você vir essa saída, espere mais um pouco e tente novamente.
"""

In [69]:
from IPython.display import Markdown
Markdown("marketing_report.md")

**Marketing Strategies for the Technological Innovation Conference**

To promote the Technological Innovation Conference and engage at least 500 potential participants, I will implement the following marketing strategies:

1. **Identify the Target Audience**: Identify the key stakeholders, including industry experts, researchers, and professionals, who would be interested in attending the conference.
2. **Create a Killer Conference Website**: Design a user-friendly website that provides all the necessary information about the conference, including the agenda, speakers, and registration details.
3. **Social Media Promotion**: Utilize social media platforms, such as Twitter, LinkedIn, and Facebook, to promote the conference and engage with potential attendees.
4. **Email Marketing**: Send targeted email campaigns to the identified audience, highlighting the benefits of attending the conference and providing updates on the agenda and speakers.
5. **Influencer Marketing**: Partner with industry influencers and thought leaders to promote the conference and attract a wider audience.
6. **Content Marketing**: Create informative and engaging content, such as blog posts, whitepapers, and webinars, to educate potential attendees about the conference and its topics.
7. **Paid Advertising**: Use targeted online advertising, such as Google Ads and LinkedIn Ads, to reach a wider audience and drive traffic to the conference website.
8. **Networking Opportunities**: Provide opportunities for attendees to network and connect with each other, such as through social media groups and online forums.
9. **Sponsorship and Partnerships**: Partner with relevant organizations and companies to sponsor the conference and provide additional value to attendees.
10. **Post-Event Follow-up**: Follow up with attendees after the conference to gather feedback and provide additional resources and information.

By implementing these marketing strategies, I am confident that we can attract at least 500 potential participants to the Technological Innovation Conference and make it a successful and engaging event.